In [1]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [5]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [6]:
ratings = [x['overall'] for x in data]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class KeySelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key=key
    def fit(self,X,y=None):
        return self
    def transform(self, X):
        return [row[self.key] for row in X]

In [8]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [47]:
pipe = Pipeline([
    ('get_reviews', KeySelector('reviewText')),
    ('vectorization', CountVectorizer(stop_words=STOP_WORDS))
])

In [33]:
pipe.fit_transform(data)[:2]

<2x76675 sparse matrix of type '<class 'numpy.int64'>'
	with 78 stored elements in Compressed Sparse Row format>

In [48]:

bag_of_words_model = Pipeline([
    ('get_reviews', KeySelector('reviewText')),
    ('vectorization', CountVectorizer(stop_words=STOP_WORDS)),
    ('estimator', GridSearchCV(Ridge(), param_grid={'alpha':np.logspace(-0.5,2,15)},
                                        n_jobs=2, verbose=1))
])
bag_of_words_model.fit(data,ratings)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed: 11.5min finished


Pipeline(memory=None,
         steps=[('get_reviews', KeySelector(key='reviewText')),
                ('vectorization',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words={'a', 'about', 'above', 'across',
                                             'a...
                                              random_state=None, solver='auto',
                                              tol=0.001),
                              iid='warn', n_jobs=2,
                              param_grid={'alpha': array([  0.31622777,   0.47705827,   0.71968567,   1.08571112,
         1.63789371,   2.47091123, 

In [50]:
# alpha = 166.81005

In [49]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score:  1.0219679125023209


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [51]:
from sklearn.feature_extraction.text import HashingVectorizer

In [58]:
estimator = Ridge(1.5)

normalized_model = Pipeline([
    ('get_reviews', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer(stop_words=STOP_WORDS)),
    ('regressor', estimator)
    
])
normalized_model.fit(data,ratings)

Pipeline(memory=None,
         steps=[('get_reviews', KeySelector(key='reviewText')),
                ('vectorizer',
                 HashingVectorizer(alternate_sign=True, analyzer='word',
                                   binary=False, decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, n_features=1048576,
                                   ngram_range=(1, 1), norm='l2',
                                   preprocessor=None,
                                   stop_words={'a', 'about', 'above'...
                                               'also', 'although', 'always',
                                               'am', 'among', 'amongst',
                                               'amount', 'an', 'and', 'another',
                                               'any', 'anyhow', 'anyone',
                                  

In [59]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score:  1.0005711836154456


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:

bigrams_model = Pipeline([
    ('text', KeySelector('reviewText')),
    ('vectorizer',TfidfVectorizer(stop_words=STOP_WORDS,ngram_range=(1,2))),
    ('estimator', Ridge(1.0))
])
bigrams_model.fit(data,ratings)

Pipeline(memory=None,
         steps=[('text', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a', 'about', '...
                                             'among', 'amongst', 'amount', 'an',
                                             'and', 'another', 'any', 'anyhow',
                                             'anyone', 'anything', 'anyway',
                                             'anywhere', 'are', ...},
                                 strip_acce

In [11]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  1.009272377709469


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [4]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [5]:
del data, ratings

NameError: name 'data' is not defined

In [6]:
import numpy as np

import gzip
import ujson as json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from spacy.lang.en import STOP_WORDS

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [int(X['overall']) for X in data_polarity]


In [8]:
pipe = Pipeline([
    ('text',KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS)),
    ('classifier',MultinomialNB())
])

In [9]:
pipe.fit(data_polarity,ratings)

Pipeline(memory=None,
         steps=[('text', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'a', 'about', '...
                                             'also', 'although', 'always', 'am',
                                             'among', 'amongst', 'amount', 'an',
                                             'and', 'another', 'any', 'anyhow',
                                             'anyone', 'anything', 'anyway',
                                

In [11]:
nb=pipe.named_steps['classifier']
log_prob=nb.feature_log_prob_

In [20]:
polarity = log_prob[0,:] - log_prob[1,:]
ind = np.argsort(polarity)


In [22]:
ind_top= np.hstack((ind[:25],ind[:-26:-1]))

In [23]:
ind_to_token= pipe.named_steps['vectorizer'].get_feature_names()
top_50=[ind_to_token[i] for i in ind_top]

In [24]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*